In [40]:
import pyteomics
import lxml
from pyteomics import mzml, mass
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from matplotlib import pyplot
import matplotlib
import seaborn as sns
import re
import math
import statsmodels
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.stats import f_oneway
from scipy.stats import pearsonr
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [2]:
df_pro=pd.read_csv(r'D:\\96_well\\csodiaq_results\\common_proteins.csv')
df_meta=pd.read_csv(r'D:\\96_well\\mzmine\\all_metabolites_unscaled.csv')

In [3]:
df_meta=df_meta.drop_duplicates(subset=['mz'],keep='first')

In [ ]:
df_pro=df_pro.iloc[:,:83]
df_pro.set_index('Unnamed: 0',inplace=True)
new_dfpro=np.log2(df_pro)
new_dfpro

In [5]:
new_dfpro.columns=[item.split('/')[-1].split('.')[0] for item in new_dfpro.columns]

In [ ]:
# apply standard scaler to the new dataframe 

ss = StandardScaler() 
df_ss = ss.fit_transform(new_dfpro)
dfpro_std=pd.DataFrame(df_ss,index=new_dfpro.index,columns=new_dfpro.columns)
dfpro_std

In [7]:
# function for oneway_ANOVA analysis 
def oneway_ANOVA(result=dfpro_std):
    pval=[]
    for i in range(len(result)):
        aa=result.iloc[i,:11].tolist()
        bb=result.iloc[i,11:23].tolist()
        cc=result.iloc[i,23:35].tolist()
        dd=result.iloc[i,35:46].tolist() 
        ee=result.iloc[i,46:58].tolist()
        ff=result.iloc[i,58:70].tolist()
        gg=result.iloc[i,70:82].tolist()
        p_value=f_oneway(aa,bb,cc,dd,ee,ff,gg)[1]
        pval.append([i,p_value])
        fl = pd.DataFrame(pval)
        fl.columns = ['number', 'p_value']
    return(fl)

In [ ]:
# make a dataframe for all p_values and corresponding protein name 

dfpro_pval=oneway_ANOVA(result=dfpro_std)
dfpro_pval['index_name']=dfpro_std.index.values.tolist()
dfpro_pval

In [9]:
len([item for item in dfpro_pval['p_value'].tolist() if item <0.05])

306

In [10]:
FDR_res=statsmodels.stats.multitest.fdrcorrection(dfpro_pval['p_value'].tolist(), alpha=0.05, method='indep', is_sorted=False)[0]


In [ ]:
# calculate the pvalues < 0.01 after FDR correction 

yuyu=[]
for item in range(len(FDR_res)):
    if FDR_res[item]== True:
        yuyu.append(item)
len(yuyu)

In [ ]:
# locate the significant proteins from df_pval dataframe, aiming to find the protein names out 

sig_pros=dfpro_pval.loc[yuyu]
sig_pros


In [ ]:

scaler=StandardScaler()
dfpro_std2=dfpro_std
proname=dfpro_std2.T.columns
k=dfpro_std2.T
dfdf_pro=ss.fit_transform(k)

dfpro_stdbyrow=pd.DataFrame(dfdf_pro.T,index=dfpro_std2.index,columns=dfpro_std2.columns)
dfpro_stdbyrow

In [ ]:
prodf=dfpro_std
#calculate the mean value of proteins from all repeats in each treatment
con=prodf.iloc[:,:11].mean(axis=1).tolist()
con_dmso=prodf.iloc[:,11:23].mean(axis=1).tolist()
DFO=prodf.iloc[:,23:35].mean(axis=1).tolist()
TORI2=prodf.iloc[:,35:46].mean(axis=1).tolist()
IDRIB=prodf.iloc[:,46:58].mean(axis=1).tolist()
N2356=prodf.iloc[:,58:70].mean(axis=1).tolist()
M7449=prodf.iloc[:,70:82].mean(axis=1).tolist()


newdict={'Con':con,'DMSO':con_dmso,'DFO':DFO,
        'TORIN2':TORI2,'ISRIB':IDRIB,'A939572':N2356,
        'MG-132':M7449}
df_mean_pro=pd.DataFrame.from_dict(newdict)
df_mean_pro.index=prodf.index
df_mean_pro  # dataframe of mean value of proteins 

In [19]:
# select significant proteins from the mean value dataframe of proteins 

dfpro_sig=df_mean_pro.loc[sig_pros['index_name'].tolist()]

In [ ]:
dfpro_sig.columns

# clustering analysis of metabolites of 96well plate starts from here

In [ ]:
newdf_meta=df_meta.set_index('mz')
newdf_meta

In [15]:
newdf_meta = newdf_meta.drop('D10', axis=1)

In [ ]:
newdf_meta

In [ ]:
# apply standard scaler to the new dataframe of metabolites 

ss = StandardScaler() 
df_sss = ss.fit_transform(newdf_meta)
df_std_meta=pd.DataFrame(df_sss,index=newdf_meta.index,columns=newdf_meta.columns)
df_std_meta

In [ ]:
# make a dataframe for all p_values and corresponding metabolites m/z  

df_pval_meta=oneway_ANOVA(result=df_std_meta)  # calculate p value of all metabolites 
df_pval_meta['index_name']=df_std_meta.index.values.tolist()
df_pval_meta

In [ ]:
len([item for item in df_pval_meta['p_value'].tolist() if item <0.05])

In [20]:
# FDR correction for all p_values of metabolites from Standard scaler less than 0.05 

FDR_res2=statsmodels.stats.multitest.fdrcorrection(df_pval_meta['p_value'].tolist(), alpha=0.01, method='indep', is_sorted=False)[0]


In [ ]:
# calculate the pvalues < 0.05 after FDR correction 
yuyu2=[]
for item in range(len(FDR_res2)):
    if FDR_res2[item]== True:
        yuyu2.append(item)
len(yuyu2)

In [ ]:
# locate the significant proteins from df_pval dataframe, aiming to find the protein names out 
sig_meta=df_pval_meta.loc[yuyu2]
sig_meta

In [ ]:

scaler=StandardScaler()
df_std_meta2=df_std_meta
metamz=df_std_meta2.T.columns
kk2=df_std_meta2.T
dfdf_meta=ss.fit_transform(kk2)

df_meta_stdbyrow=pd.DataFrame(dfdf_meta.T,index=df_std_meta2.index,columns=df_std_meta2.columns)
df_meta_stdbyrow


In [ ]:

#calculate the mean value of metabolites from all repeats in each treatment
metadf=df_std_meta
con_meta=metadf.iloc[:,:11].mean(axis=1).tolist()
dmso_meta=metadf.iloc[:,11:23].mean(axis=1).tolist()
DFO_meta=metadf.iloc[:,23:35].mean(axis=1).tolist()
TORI2_meta=metadf.iloc[:,35:46].mean(axis=1).tolist()
IDRIB_meta=metadf.iloc[:,46:58].mean(axis=1).tolist()
N2356_meta=metadf.iloc[:,58:70].mean(axis=1).tolist()
M7449_meta=metadf.iloc[:,70:82].mean(axis=1).tolist()


newdict_meta={'Con':con_meta,'DMSO':dmso_meta,'DFO':DFO_meta,
        'TORIN2':TORI2_meta,'ISRIB':IDRIB_meta,'A939572':N2356_meta,
        'MG-132':M7449_meta}

df_mean_meta=pd.DataFrame.from_dict(newdict_meta)
df_mean_meta.index=metadf.index
df_mean_meta  # dataframe of mean value of proteins 

In [25]:
# select significant proteins from the mean value dataframe of proteins 

dfmeta_sig=df_mean_meta.loc[sig_meta['index_name'].tolist()]

In [ ]:
df_mean_pro

In [72]:
dfdf=df_mean_meta
#dfdf=df_mean_pro

list1 = dfdf['DMSO'].tolist()
list2 = dfdf['DFO'].tolist()
list3 = dfdf['TORIN2'].tolist()
list4 = dfdf['ISRIB'].tolist()
list5 = dfdf['A939572'].tolist()
list6 = dfdf['MG-132'].tolist()
list_con = dfdf['Con'].tolist()

DMSO = [x - y for x, y in zip(list1, list_con)]
DFO = [x - y for x, y in zip(list2, list_con)]
TORIN2 = [x - y for x, y in zip(list3, list_con)]
ISRIB = [x - y for x, y in zip(list4, list_con)]
A939572 = [x - y for x, y in zip(list5, list_con)]
MG_132 = [x - y for x, y in zip(list6, list_con)]

lists=[DMSO,DFO,TORIN2,ISRIB,A939572,MG_132]

In [77]:


def pearson_correlation(list1, list2):
    correlation_coefficient = np.corrcoef(list1, list2)[0, 1]
    return correlation_coefficient

def plot_jointplot(resultx=DMSO,resulty=DFO,xlabel='DMSO vs Con', ylabel='DFO vs Con'):
    plt.rcParams['font.family'] = 'Arial'
    plt.figure(figsize=(4,4))
    g=sns.jointplot(x=resultx, 
                      y=resulty,
                      #xlim = (13,26), ylim = (13,26),
                      kind="reg", truncate=False,
                      color="deepskyblue", height=6)
                      #color="purple", height=6)
    g.ax_joint.tick_params(labelsize=22)
    g.set_axis_labels(xlabel, ylabel, fontsize=22)
    plt.savefig(r'D:\\project7_SMAD\\SI\\{}.svg'.format(xlabel+"meta"+ylabel),dpi=800,bbox_inches='tight')
    print(pearson_correlation(resultx, resulty))
    

In [78]:
# calculate pearson correlation
correlation_matrix = np.zeros((len(lists), len(lists)))

for i in range(len(lists)):
    for j in range(i+1, len(lists)):
        corr, _ = pearsonr(lists[i], lists[j])
        #corr=np.corrcoef(lists[i], lists[j])[0, 1]
        correlation_matrix[i][j] = corr
        correlation_matrix[j][i] = corr
        
for i in range(len(lists)):
    corr,_=pearsonr(lists[i], lists[i])
    correlation_matrix[i][i] = corr



In [ ]:

plt.figure(figsize=(8, 6))
plt.rcParams['font.family'] = 'Arial'
sns.heatmap(correlation_matrix, annot=True, cmap='viridis', fmt=".3f",
            xticklabels=['DMSO','DFO','TORIN2','ISRIB','A939572','MG_132'],
            yticklabels=['DMSO','DFO','TORIN2','ISRIB','A939572','MG_132'])
#plt.title("Pearson Correlation of Proteome")
#plt.xlabel("List Index")
#plt.ylabel("List Index")
plt.savefig(r'D:\\project7_SMAD\\SI\\meta_corre_heatmap.svg',dpi=800,bbox_inches='tight')

plt.show()

In [ ]:
plot_jointplot(resultx=TORIN2,resulty=MG_132,xlabel='TORIN2 vs Con', ylabel='MG-132 vs Con')

In [ ]:
plot_jointplot(resultx=TORIN2,resulty=DMSO,xlabel='TORIN2 vs Con', ylabel='DMSO vs Con')